# Extrator de dados do Sweet Coffee Week - Natal (2021)

## Importação de libs

In [4]:
import mechanize
import pandas as pd
import bs4
import requests
from bs4 import BeautifulSoup
import re
from concurrent import futures
from urllib.parse import urlparse

## Extração de links das páginas internas das cafeterias

In [5]:
brw = mechanize.Browser()

In [6]:
pag = brw.open('https://www.sweetcoffeeweek.com.br/')

In [7]:
soup = BeautifulSoup(pag.read())

In [8]:
a_urls = soup.find_all('a', {'href': re.compile(r'https://www\.sweetcoffeeweek\.com\.br/\w+')})[:-1]
urls = list(set([link['href'] for link in a_urls]))

In [ ]:
urls

## Extração de dados de menu das cafeterias

In [ ]:
def baixar_dados_cafeterias(url):
    print(url, '⏳')
    page = brw.open(url)
    soup = BeautifulSoup(page.read())
    print(url, '✅')
    return soup


with futures.ThreadPoolExecutor(max_workers=4) as executor:
    threads = [executor.submit(baixar_dados_cafeterias, url) for url in urls]
    futures.wait(threads, timeout=120)  

In [89]:
def processar_cafeteria(soup):
    nome = soup.find_all('h4')[1].text
    prato = soup.find_all('div', **{'data-testid': 'columns'})[1].find_all('div')[7].text.replace('\n', '')
    dicio = {
        'nome': nome,
    }
    pos = 0
    while r:=re_pratos.search(prato, pos=pos):
        pos = r.end()+1
        dicio |= {k:v.strip() for k,v in r.groupdict().items() if v is not None}
        
    return dicio

In [91]:
dados_cafeterias = []
re_pratos = re.compile(r'(?P<doce>(?<=DOCE:).+(?=SALGADO:))|(?P<salgado>(?<=SALGADO:).+(?=BEBIDA:))|(?P<bebida>(?<=BEBIDA:).+$)', 
                       re.MULTILINE)

for t in threads:
    soup = t.result()
    dados_cafeterias.append(processar_cafeteria(soup))

## Resultado da extração

In [92]:
df_cafeterias = pd.DataFrame(dados_cafeterias)

In [95]:
df_cafeterias

,nome,doce,salgado,bebida
0,BOCADITOS CONFEITARIA,Beijo da Boca: Entremet bem tropical e alegre ...,Sebosão Gourmet da Boca: Sebosão com pão artes...,Café espresso com chantilly colorido e purpuri...
1,PANEER PATISSERIE,Brioche na chapa com manteiga da terra e cocad...,Brioche na chapa com manteiga da terra e carne...,Frapê com chantilly.
2,CHEF FIT'S,Crepe de cartola com sorvete de creme zero açú...,Coxinha de frango no espeto com massa de macax...,Café coado ou água.
3,FRAN'S CAFÉ\nCIDADE JARDIM & PARTAGE NORTE SHO...,Croissant de Doce de Leite;,Porção de mini Pão de Queijo;,Espresso ou Mini Francino de Doce de Leite.
4,DA TERRA COMIDA REGIONAL,Tapioca de coco com leite condensado;,Cuscuz com carne de sol na nata;,Café puro OU Café com leite OU Cajuina 250ml O...
5,DOCE LELÊ,Cheesecake desconstruída;,Pão delícia com carne de sol cremosa;,Cappuccino ou água (água para delivery).
6,DUART'S CONFEITARIA,Bolo de coco com compota de abacaxi e brigadei...,Dadinho de tapioca com carne de sol acompanhad...,Café coado ou refrigerante pet 200ml.
7,CECÍLIA MINDÊLO,Filhós pedacinho do céu com sorvete e melaço d...,Torrada amostrada de carne de sol na nata com ...,Café caicoense e rosquinha de queijo para acom...
8,VERYSUGAR,"Cocada de colher quentinha, servida com sorvet...",Coxinha(120g) de camarão ao queijo de manteiga;,Smoothie de cajá ou café espresso.
9,LULU CAFÉ E PATISSERIE,"Sorvete de creme, calda de goiabada quente e r...",Croquete de macaxeira com carne de sol na nata...,Café com leite.
